In [1]:
import pandas as pd
import pathlib
import numpy as np
from pprint import pprint
from functools import partial
import json
import random

from src.data import datasets, utils, Dataset, RawDataset
from src.data.datasets import build_dataset_dict, load_dataset, fetch_and_unpack
from src.data.fetch import fetch_text_file, hash_file
from src.data.utils import list_dir, head_file, normalize_labels, read_space_delimited, reservoir_sample
from src.paths import interim_data_path, raw_data_path, processed_data_path

In [2]:
%load_ext autoreload
%autoreload 2

# Adding and Processing the Yelp Dataset



### Adding to the raw dataset list
A raw dataset is really just a list of files (and their associated hash values) and metadata that is later processed into a usable dataset. 

1. Assemble the raw data files.
2. Generate hashes to check the validity of these files
3. Add LICENSE and DESCR (description) metadata to the dataset


In [3]:
dataset_name='yelp'

The yelp dataset is free to use under a reasonable license agreement, but you need to register to download a copy.
https://www.yelp.com/dataset/download


Once we have followed the instructions at the above link, we should have a local copy of the tarfile

By default, we store raw data in `src.paths.raw_data_path` (by default: `data/raw`)

In [4]:
from src.paths import raw_data_path
from src.data.utils import list_dir

list_dir(raw_data_path)

['.gitkeep', 'yelp.readme', 'yelp_dataset.tar.gz']

If you have an independent way of verifying the hashes, do so. Otherwise, you can compute them the first time using `build_dataset_dict`. Note, this can also download from a URL if no extra verification is required

In [5]:
help(build_dataset_dict)

Help on function build_dataset_dict in module src.data.datasets:

build_dataset_dict(hash_type='sha1', hash_value=None, url=None, name=None, file_name=None, from_txt=None)
    Build a raw dataset dictionary entry for a file.
    
    This will fetch the file if `url` is specified.
    
    If `hash_value` is specified, the file hash will be checked.
    Otherwise, the hash will be computed and returned.
    
    hash_type: {'sha1', 'md5', 'sha256'}
    hash_value: string or None
        Hash to use when validating the specified file.
        if None, the hash will be computed from downloaded file
    file_name: string or None
        Name of downloaded file. If None, will be the last component of the URL
    url: string
        If speficied, the file will be fetched from this URL
    from_txt: string
        contents of file to create.
        One of `url` or `from_txt` must be specified
    name: string
        this field can be used to indicate the type of datafile being downloaded.


In [6]:
rawdata = RawDataset(dataset_name=dataset_name)
RawDataset.add_file('yelp_dataset.tar.gz', hash_type='sha1')
RawDataset.add_url('https://...', hash_type='sha1', fetch=True)
rawdata.metadata['descr'] = yelp_readme
rawdata.metadata['license'] = yelp_license

RawDataset.set_process_function(func)

rawdata.save_json(dataset_name, 'dataset.json')

def load_dataset(rawdata, **opts):
    rawdata.fetch()
    rawdate.unpack()
    ds = rawdata.process(**opts)
    return ds

ds.dump(file_stub)

raw_datasets = [
    {
        'dataset_name':'yelp',
        'ds_opts':{'num_reviews':10000},
        'alg_name':est,
        'alg_opts':{},
        'suffix': None
    }
]

for ds_stub, opts in raw_datasets.items():
    if suffix is None:
        ds_stub = f'{dataset_name}_{alg_name}'
    else:
        ds_stub = f'{dataset_name}_{alg_name}_{suffix}'
    # check if ds_stub already exists on disk
    ## if exists, check metadata, including ds_opts, alg_opts. If different, die with error
    ds = load_dataset(opts['dataset_name'], **opts['ds_opts'])
    ds_out = pipeline.transform(ds.data)
    ds_out.dump(ds_stub)

Dataset.load(file_stub)

{'name': None,
 'file_name': 'yelp_dataset.tar.gz',
 'hash_type': 'sha1',
 'hash_value': '096ac5ced8a9229ecc5116e77b6be8d8f90fdacb'}

The **name** field can be used to indicate the type of datafile being downloaded. Usually, this is just informational. However, if you specify names `DESCR` or `LICENSE`, the downloaded (text) file will be used as the dataset description and license text, respectively.

Usually you will want to give these unique names, so they don't clash with other downloaded files. (e.g. "LICENSE.txt" is a terrible name to use). We use the **file_name** option for this:

### Adding a License and Description

Every dataset should have an associated **license** and **description**. These are stored as `LICENSE` and `DESCR` attributes respectively.

There are 3 typical ways to add license and description text:
* Download textfiles from somewhere on the Internet, and using the `name` parameter in `build_dataset_dict` to tag them as either `LICENSE` or `DESCR`
* Create a `{dataset_name}.license`, or `{dataset_name}.readme` in `src/data`. When a dataset matching this
name is created, this information will automatically be used, or
* specify the text directly using `from_txt` in `build_dataset_dict()`

#### Adding A License File

In this case, the license is available from https://s3-media1.fl.yelpcdn.com/assets/srv0/engineering_pages/06cb5ad91db8/assets/vendor/yelp-dataset-agreement.pdf

we can write this to a textfile directly

In [7]:
%%writefile ../src/data/yelp.license
YELP DATASET TERMS OF USE

Last Updated: July 26, 2018
This document governs the terms under which you may access and use the data that Yelp
makes available for download through this website (or made available by other means) for
academic purposes (the “Data”). This document incorporates the terms of the following
additional document, including all future amendments or modifications thereto (collectively, and
together with this document, the “Data Agreement” ):

Yelp Terms of Service:

By accessing or using the Data, you agree to be bound by the Data Agreement and represent
that the contact information you provide to Yelp is correct. If you access or use the Data on
behalf of a university, school, or other entity, you represent that you have authority to bind such
entity and its affiliates to the Data Agreement and that it is fully binding upon them. In such
case, the term “you” and “your” will refer to such entity and its affiliates. If you do not have
authority, or if you do not agree with the terms of the Data Agreement, you may not access or
use the Data. You should read and keep a copy of each component of the Data Agreement for
your records. In the event of a conflict among them, the terms of this document will control.

1. Purpose

The Data is made available by Yelp Inc. (“Yelp”) to enable you to access valuable
local information to develop an academic project as part of an ongoing course of study. With this
in mind, Yelp reserves the right to continually review and evaluate all uses of the Data provided
under the Data Agreement.

2. Changes

Yelp reserves the right to modify or revise the Data Agreement at any time. If the
change is deemed to be material and it is foreseeable that such change could be adverse to
your interests, Yelp will provide you notice of the change to this Data Agreement by sending you
an email to the email you provided to Yelp. Your continued use of the Data after the notice of
material change will constitute your acceptance of and agreement to such changes. 

If YOU DO
NOT WISH TO BE BOUND TO ANY NEW TERMS, YOU MUST TERMINATE THE DATA
AGREEMENT BY IMMEDIATELY CEASING USE OF THE DATA AND DELETING IT FROM
ANY SYSTEMS OR MEDIA.

3. License

Subject to the terms set forth in the Data Agreement (specifically the restrictions set
forth in Section 4 below), Yelp grants you a royalty-free, non-exclusive, revocable,
non-sublicensable, non-transferable, fully paid-up right and license during the Term to use,
access, and create derivative works of the Data in electronic form for academic purposes only.
You may not use the Data for any other purpose without Yelp’s prior written consent. You
acknowledge and agree that Yelp may request information about, review, audit, and/or monitor
your use of the Data at any time in order to confirm compliance with the Data Agreement.
Nothing herein shall be construed as a license to use Yelp’s registered trademarks or service
marks, or any other Yelp branding.

4. Restrictions

You agree that you will not, and will not encourage, assist, or enable others to:
A. display, perform, or distribute any of the Data, or use the Data to update or create
your own business listing information (i.e. you may not publicly display any of the Data to any
third party, especially reviews and other user generated content, as this is a private data set
challenge and not a license to compete with or disparage with Yelp);
B. use the Data in connection with any commercial purpose;
C. use the Data in any manner or for any purpose that may violate any law or regulation,
or any right of any person including, but not limited to, intellectual property rights, rights of
privacy and/or rights of personality, or which otherwise may be harmful (in Yelp's sole
discretion) to Yelp, its providers, its suppliers, end users of this website, or your end users;
D. use the Data on behalf of any third party without Yelp’s consent;
E. create, redistribute or disclose any summary of, or metrics related to, the Data (e.g.,
the number of reviewed business included in the Data and other statistical analysis) to any third
party or on any website or other electronic media not expressly covered by this Agreement, this
provision however, excludes any disclosures necessary for academic purposes, including
without limitation the publication of academic articles concerning your use of the Data;
F. use the Data in a manner that is competitive in nature with Yelp;
G. display Data in a manner that could reasonably imply an endorsement, relationship or
affiliation with or sponsorship between you or a third party and Yelp, other than your permitted
use of the Data under the terms of the Data Agreement;
H. rent, lease, sell, transfer, assign, or sublicense, any part of the Data;
I. modify, rate, rank, review, vote or comment on, or otherwise respond to the content
contained in the Data;
J. display the Data or publicly communicate in any way, or on any site, in a manner that
disparages Yelp or its products or services, or infringes any Yelp intellectual property or other
rights;
K. use the Data in a manner that could reasonably be interpreted to suggest that Yelp is
the author or entity that is responsible, in whole or in part, for the creation or development of any
Data or that such Data represents the views of Yelp; or
L. use the Data for any purpose prohibited by law.

5. Ownership

As between you and Yelp, the Data and any derivative works you create from the
Data, and all intellectual property rights contained in the foregoing, are and will at all times
remain the sole and exclusive property of Yelp and are protected by applicable intellectual
property laws and treaties (whether those rights happen to be registered or not, and wherever in
the world those rights may exist), or as otherwise set forth in the contest rules where the various
submitted solutions must be made available under a specified open source license, such as the
MIT License.

6. Indemnity

You agree that your use of the Data is at your own risk and you agree to hold
harmless, defend (subject to Yelp's right to participate with counsel it selects) and indemnify
Yelp and its subsidiaries, affiliates, officers, agents, employees and suppliers from and against
any and all claims, damages, liabilities, costs and fees (including reasonable attorneys’ fee)
arising from, or in any way related to your or your end users’ use or implementation of the Data.
You will not agree to any settlement that imposes any obligation on Yelp without Yelp's prior
consent.
                  
7. No Warranties by Yelp; No Entitlement to Support from Yelp

THE DATA IS PROVIDED
“AS IS”, “WITH ALL FAULTS” AND “AS AVAILABLE” WITHOUT WARRANTY, OF ANY KIND
AND AT YOUR SOLE RISK. EXCEPT TO THE MAXIMUM EXTENT REQUIRED BY
APPLICABLE LAW, YELP DISCLAIMS ALL WARRANTIES, REPRESENTATIONS,
CONDITIONS, AND DUTIES, WHETHER EXPRESS, IMPLIED OR STATUTORY,
REGARDING THE DATA, INCLUDING, WITHOUT LIMITATION, ANY AND ALL IMPLIED
WARRANTIES OF MERCHANTABILITY, ACCURACY, RESULTS OF USE, RELIABILITY,
FITNESS FOR A PARTICULAR PURPOSE, TITLE, INTERFERENCE WITH QUIET
ENJOYMENT AND NON-INFRINGEMENT OF THIRD-PARTY RIGHTS. FURTHER, YELP
DISCLAIMS ANY WARRANTY THAT YOUR USE OF THE DATA WILL BE UNINTERRUPTED,
SECURE, TIMELY OR ERROR FREE. FOR THE AVOIDANCE OF DOUBT, YOU
ACKNOWLEDGE AND AGREE THAT THE DATA AGREEMENT DOES NOT ENTITLE YOU
TO ANY SUPPORT FOR THE DATA. NO ADVICE OR INFORMATION, WHETHER ORAL OR
IN WRITING, OBTAINED BY YOU FROM YELP WILL CREATE ANY WARRANTY NOT
EXPRESSLY STATED IN THE DATA AGREEMENT.

8. Limitation of Liability

THE DATA IS BEING PROVIDED FREE OF CHARGE.
ACCORDINGLY, YOU AGREE THAT YELP SHALL HAVE NO LIABILITY ARISING FROM OR
BASED ON YOUR USE OF THE DATA. REGARDLESS OF WHETHER ANY REMEDY SET
FORTH HEREIN FAILS OF ITS ESSENTIAL PURPOSE OR OTHERWISE, AND EXCEPT FOR
BODILY INJURY, IN NO EVENT SHALL YELP OR ITS SUBSIDIARIES, AFFILIATES,
OFFICERS, AGENTS, EMPLOYEES AND SUPPLIERS BE LIABLE TO YOU OR TO ANY
THIRD PARTY UNDER ANY TORT, CONTRACT, NEGLIGENCE, STRICT LIABILITY OR
OTHER LEGAL OR EQUITABLE THEORY FOR ANY LOST PROFITS, LOST OR
CORRUPTED DATA, COMPUTER FAILURE OR MALFUNCTION, INTERRUPTION OF
BUSINESS, OR OTHER SPECIAL, INDIRECT, INCIDENTAL OR CONSEQUENTIAL
DAMAGES OF ANY KIND ARISING OUT OF THE USE OR INABILITY TO USE THE DATA,
EVEN IF YELP HAS BEEN ADVISED OF THE POSSIBILITY OF SUCH LOSS OR DAMAGES
AND WHETHER OR NOT SUCH LOSS OR DAMAGES ARE FORESEEABLE. ANY CLAIM
ARISING OUT OF OR RELATING TO THE DATA AGREEMENT MUST BE BROUGHT WITHIN
(1) YEAR AFTER THE OCCURRENCE OF THE EVENT GIVING RISE TO SUCH CLAIM. IF
SUCH CLAIM IS NOT FILED, THEN THAT CLAIM IS PERMANENTLY BARRED. THIS
APPLIES TO YOU AND YOUR SUCCESSORS, AND TO YELP AND ITS SUCCESSORS.
NOTWITHSTANDING THE FOREGOING, SINCE THIS LICENSE IS PROVIDED TO YOU AT
NO CHARGE, YELP’S MAXIMUM LIABILITY UNDER THIS DATA AGREEMENT SHALL NOT,
IN ANY EVENT, EXCEED US$50.00.

9. Limited Relationship
                  
Yelp and You are, and will remain, independent contractors, and
nothing in the Data Agreement will be construed as creating an employer-employee
relationship, partnership or joint venture. Although you are permitted to publicize your use of the
Data, you agree not to make any other statements, without the prior written consent of Yelp,
implying a different kind of relationship between you and Yelp, including any implied
endorsement by Yelp. You do not have any authority of any kind to bind Yelp in any respect
whatsoever.

10. Term and Termination
                  
This Data Agreement is effective as of the date you download or
otherwise access the Data (“Effective Date” ) and shall continue in full force and effect for a term
of twelve (12) months from the Effective Date, unless earlier terminated by the parties or expires
in accordance with this Section 11 (the “Term”). Either party may immediately terminate this
Data Agreement, for any reason or for no reason, by providing written notice to the other party.
Yelp will provide notice of termination to the email account you provided to Yelp during
registration and termination will be effective upon delivery of the email notice. Yelp reserves the
right, in its sole discretion (for any reason or for no reason) and at any time without notice to
you, to change, suspend or discontinue the Data and/or suspend or terminate your further
access to the Data. Any termination of the Data Agreement will also immediately terminate the
licenses granted to you hereunder. Upon any termination of the Data Agreement, you will
promptly: (i) delete and remove all Data from any location, including any web pages, scripts,
widgets, applications and any other software in your possession or under your control; (ii)
destroy and remove from all computers, hard drives, networks and other storage media in your
possession or under your control all copies of any Data; and (iii) upon Yelp’s request, certify in
writing to Yelp that such actions have been taken.

11. Miscellaneous
                  
The Data Agreement encompasses the entire agreement between you and
Yelp regarding the subject matter discussed therein. The Data Agreement, and any disputes
arising from or relating to the interpretation thereof, will be governed by and construed under the
laws of the State of California without regard to its conflict of law provisions. You agree to
personal jurisdiction by and venue in the state and federal courts of the State of California, City
of San Francisco. The failure of Yelp to exercise or enforce any right or provision of the Data
Agreement will not constitute a waiver of such right or provision. The failure of either party to
exercise in any respect any right provided for herein will not be deemed a waiver of any further
rights hereunder. If any provision of the Data Agreement is found to be unenforceable or invalid,
that provision will be replaced with terms that most closely match the intent of the provision that
is not enforceable to the minimum extent necessary so that the remaining Data Agreement will
otherwise remain in full force and effect and enforceable. The Data Agreement is not
assignable, transferable or sublicensable, in whole or in part, by you except with Yelp's prior
written consent. Any attempt to do so is void. Yelp may assign the Data Agreement, in whole or
in part, at any time with or without notice to you. The section titles in the Data Agreement are for
convenience only and have no legal or contractual effect.

12. Survival 

Sections 4 through 13 will survive any expiration or termination of this Data
Agreement for any reason.

13.  Contact and Violations
                  
Please contact Yelp with any questions regarding the Data
Agreement. Please report any violations of the Data Agreement couvidat@yelp.com.


Overwriting ../src/data/yelp.license


The other thing we can do is just save the text to a string, and add it using `from_txt`:

In [8]:
readme_txt = '''
Yelp Dataset JSON

Each file is composed of a single object type, one JSON-object per-line.

Take a look at some examples to get you started: https://github.com/Yelp/dataset-examples.

Note: the follow examples contain inline comments, which are technically not valid JSON. This is done here to simplify the documentation and explaining the structure, the JSON files you download will not contain any comments and will be fully valid JSON.
business.json

Contains business data including location data, attributes, and categories.

{
    // string, 22 character unique string business id
    "business_id": "tnhfDv5Il8EaGSXZGiuQGg",

    // string, the business's name
    "name": "Garaje",

    // string, the neighborhood's name
    "neighborhood": "SoMa",

    // string, the full address of the business
    "address": "475 3rd St",

    // string, the city
    "city": "San Francisco",

    // string, 2 character state code, if applicable
    "state": "CA",

    // string, the postal code
    "postal code": "94107",

    // float, latitude
    "latitude": 37.7817529521,

    // float, longitude
    "longitude": -122.39612197,

    // float, star rating, rounded to half-stars
    "stars": 4.5,

    // interger, number of reviews
    "review_count": 1198,

    // integer, 0 or 1 for closed or open, respectively
    "is_open": 1,

    // object, business attributes to values. note: some attribute values might be objects
    "attributes": {
        "RestaurantsTakeOut": true,
        "BusinessParking": {
            "garage": false,
            "street": true,
            "validated": false,
            "lot": false,
            "valet": false
        },
    },

    // an array of strings of business categories
    "categories": [
        "Mexican",
        "Burgers",
        "Gastropubs"
    ],

    // an object of key day to value hours, hours are using a 24hr clock
    "hours": {
        "Monday": "10:00-21:00",
        "Tuesday": "10:00-21:00",
        "Friday": "10:00-21:00",
        "Wednesday": "10:00-21:00",
        "Thursday": "10:00-21:00",
        "Sunday": "11:00-18:00",
        "Saturday": "10:00-21:00"
    }
}

review.json

Contains full review text data including the user_id that wrote the review and the business_id the review is written for.

{
    // string, 22 character unique review id
    "review_id": "zdSx_SD6obEhz9VrW9uAWA",

    // string, 22 character unique user id, maps to the user in user.json
    "user_id": "Ha3iJu77CxlrFm-vQRs_8g",

    // string, 22 character business id, maps to business in business.json
    "business_id": "tnhfDv5Il8EaGSXZGiuQGg",

    // integer, star rating
    "stars": 4,

    // string, date formatted YYYY-MM-DD
    "date": "2016-03-09",

    // string, the review itself
    "text": "Great place to hang out after work: the prices are decent, and the ambience is fun. It's a bit loud, but very lively. The staff is friendly, and the food is good. They have a good selection of drinks.",

    // integer, number of useful votes received
    "useful": 0,

    // integer, number of funny votes received
    "funny": 0,

    // integer, number of cool votes received
    "cool": 0
}

user.json

User data including the user's friend mapping and all the metadata associated with the user.

{
    // string, 22 character unique user id, maps to the user in user.json
    "user_id": "Ha3iJu77CxlrFm-vQRs_8g",

    // string, the user's first name
    "name": "Sebastien",

    // integer, the number of reviews they've written
    "review_count": 56,

    // string, when the user joined Yelp, formatted like YYYY-MM-DD
    "yelping_since": "2011-01-01",

    // array of strings, an array of the user's friend as user_ids
    "friends": [
        "wqoXYLWmpkEH0YvTmHBsJQ",
        "KUXLLiJGrjtSsapmxmpvTA",
        "6e9rJKQC3n0RSKyHLViL-Q"
    ],

    // integer, number of useful votes sent by the user
    "useful": 21,

    // integer, number of funny votes sent by the user
    "funny": 88,

    // integer, number of cool votes sent by the user
    "cool": 15,

    // integer, number of fans the user has
    "fans": 1032,

    // array of integers, the years the user was elite
    "elite": [
        2012,
        2013
    ],

    // float, average rating of all reviews
    "average_stars": 4.31,

    // integer, number of hot compliments received by the user
    "compliment_hot": 339,

    // integer, number of more compliments received by the user
    "compliment_more": 668,

    // integer, number of profile compliments received by the user
    "compliment_profile": 42,

    // integer, number of cute compliments received by the user
    "compliment_cute": 62,

    // integer, number of list compliments received by the user
    "compliment_list": 37,

    // integer, number of note compliments received by the user
    "compliment_note": 356,

    // integer, number of plain compliments received by the user
    "compliment_plain": 68,

    // integer, number of cool compliments received by the user
    "compliment_cool": 91,

    // integer, number of funny compliments received by the user
    "compliment_funny": 99,

    // integer, number of writer compliments received by the user
    "compliment_writer": 95,

    // integer, number of photo compliments received by the user
    "compliment_photos": 50
}

checkin.json

Checkins on a business.

{
    // nested object of the day of the week with key of
    // the hour (using a 24hr clock) with the count of checkins
    // for that hour (e.g. 14:00 - 14:59).
    "time": {
        "Wednesday": {
            "14:00": 2,
            "16:00": 1,
            "2:00": 1,
            "0:00": 1
        },
        "Sunday": {
            "16:00": 8,
            "14:00": 3,
            "15:00": 3,
            "13:00": 1,
            "18:00": 2,
            "23:00": 1,
            "21:00": 1,
            "17:00": 2
        },
        "Friday": {
            "16:00": 1,
            "13:00": 1,
            "11:00": 2,
            "23:00": 2
        },
    },

    // string, 22 character business id, maps to business in business.json
    "business_id": "tnhfDv5Il8EaGSXZGiuQGg"
}

tip.json

Tips written by a user on a business. Tips are shorter than reviews and tend to convey quick suggestions.

{
    // string, text of the tip
    "text": "Secret menu - fried chicken sando is da bombbbbbb Their zapatos are good too.",

    // string, when the tip was written, formatted like YYYY-MM-DD
    "date": "2013-09-20",

    // integer, how many likes it has
    "likes": 172,

    // string, 22 character business id, maps to business in business.json
    "business_id": "tnhfDv5Il8EaGSXZGiuQGg",

    // string, 22 character unique user id, maps to the user in user.json
    "user_id": "49JhAJh8vSQ-vM4Aourl0g"
}

photo.json

Contains photo data including the caption and classification (one of "food", "drink", "menu", "inside" or "outside").

{
    // string, 22 character unique photo id
    "photo_id": "_nN_DhLXkfwEkwPNxne9hw",


    // string, 22 character business id, maps to business in business.json
    "business_id" : "tnhfDv5Il8EaGSXZGiuQGg",

    // string, the photo caption, if any
    "caption" : "carne asada fries",

    // string, the category the photo belongs to, if any
    "label" : "food"
}
'''


Next, we combine the complete set of files into a URL list and use this to build our json file entry. In our case, just the 1 file

In [9]:
url_list = [rawdata]; url_list


[{'name': None,
  'file_name': 'yelp_dataset.tar.gz',
  'hash_type': 'sha1',
  'hash_value': '096ac5ced8a9229ecc5116e77b6be8d8f90fdacb'}]

In [10]:
url_list += [build_dataset_dict(from_txt=readme_txt, 
                                file_name=f'{dataset_name}.readme', name='DESCR')]

### datasets.json
The current list of datasets is stored in `datasets.json`, by default in `src/data/`

We can add the url list using `add_datasets_by_urllist`

In [11]:
newds_dict = datasets.add_dataset_by_urllist(dataset_name, url_list)

See that a generic `load_function` (`new_dataset`) has been used to process the data. This does nothing more than populates the DESCR and LICENSE fields (if possible), creating an otherwise empty `Dataset` object

In [12]:
dset = load_dataset(dataset_name)

In [13]:
print(dset.LICENSE)

YELP DATASET TERMS OF USE

Last Updated: July 26, 2018
This document governs the terms under which you may access and use the data that Yelp
makes available for download through this website (or made available by other means) for
academic purposes (the “Data”). This document incorporates the terms of the following
additional document, including all future amendments or modifications thereto (collectively, and
together with this document, the “Data Agreement” ):

Yelp Terms of Service:

By accessing or using the Data, you agree to be bound by the Data Agreement and represent
that the contact information you provide to Yelp is correct. If you access or use the Data on
behalf of a university, school, or other entity, you represent that you have authority to bind such
entity and its affiliates to the Data Agreement and that it is fully binding upon them. In such
case, the term “you” and “your” will refer to such entity and its affiliates. If you do not have
authority, or if you do not agre

In [14]:
len(dset.DESCR)

7282

Now, we haven't actually done anything with this raw dataset yet, other than download the raw data and set the license/description metadata.


In [15]:
type(dset.data), type(dset.target)

(NoneType, NoneType)

## Processing the data
The next step is to write an importer function that actually processes the data we will be using for this dataset - in our case, `process_yelp_data()`

The important things to generate are `data` and `target` entries. A `metadata` is optional, but recommended if you want to save additional information about the dataset.

Usually, this functionality gets bundled up into a function and added to `localdata.py`. The name (and module containing) this functionality is added to the appropriate entry in `datasets.py`


In [16]:
# Unpack the file
untar_dir = fetch_and_unpack(dataset_name)


In [17]:
untar_dir

PosixPath('/home/ava00088/src/devel/text_embedding/data/interim/yelp')

In [18]:
list_dir(untar_dir)

['._Dataset_Challenge_Dataset_Agreement.pdf',
 '._Yelp_Dataset_Challenge_Round_12.pdf',
 'Dataset_Challenge_Dataset_Agreement.pdf',
 'yelp.readme',
 'yelp_academic_dataset_business.json',
 'yelp_academic_dataset_checkin.json',
 'yelp_academic_dataset_photo.json',
 'yelp_academic_dataset_review.json',
 'yelp_academic_dataset_tip.json',
 'yelp_academic_dataset_user.json',
 'Yelp_Dataset_Challenge_Round_12.pdf']

In [19]:
biz_file = untar_dir / 'yelp_academic_dataset_business.json'
review_file = untar_dir / 'yelp_academic_dataset_review.json'

biz_file.exists() and review_file.exists()

True

Recall, our DESCR gives us a complete discription of the data format

In [20]:
def yelp_json_to_df(filename):
    """Convert a yelp JSON file to a dataframe
    
    Yelp data consists of one json-object per line.
    
    Parameters
    ----------
    
    Returns
    -------
    dataframe corresponding to the file
    """
    with open(filename, 'r') as f:
        data = f.readlines()

    # remove the trailing CR from each line
    data = map(lambda x: x.rstrip(), data)

    # Pack it up as a json list
    data_json_str = "[" + ','.join(data) + "]"

    # and load it into pandas
    df = pd.read_json(data_json_str)
    return df

In [21]:
%%time
k = 100000
sample = reservoir_sample(review_file, k)
data_json_str = "[" + ','.join(sample) + "]"
df = pd.read_json(data_json_str)

CPU times: user 30.9 s, sys: 4.03 s, total: 34.9 s
Wall time: 52.3 s


In [22]:
df.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,TmIucPRTngutrjDpC0nXgQ,1,2018-03-20,0,JNVprSkQjHfOwJMKfZWkNA,5,I don't know where to begin! I came in with a ...,1,6sGFoSoU7Jr0EbgL1EK3xQ
1,M-DIwf9-S3FqA4zI9OCAbQ,9,2018-02-20,7,V0Qjge4QgBeXSomrAnPTdQ,2,They say good things come to those who wait. T...,13,EXKDLuYoZUKUNNNYA7TlGQ
2,mz9ltimeAIy2c2qf5ctljw,0,2015-01-03,0,63NDxdKFtLejDzius77kXA,4,What a great experience! Visually a very stunn...,0,6NYDXzkyYW3O2zYjzPRMsg
3,ukov0vX_X5a76CiVaNWKvw,0,2015-01-26,0,z_Gl5bCD5xiAV5yMyi1VJg,1,I don't understand this place. This is a horri...,4,7jev9oqE9jawRKXAb7Vq_g
4,6q7TLSWByKGeIkt1HYBUgw,0,2017-01-11,0,EcON0QbB79S-Ws107hdYdw,5,"Black Bear Diner is great! Awesome food, large...",0,uMqUy9ELjvhiW6_jD47Xaw


This seems to work, so let's wrap this functionality up into a processing function.

By convention, a `process_*` function takes a `dataset_name`, and any other options that may be useful for reading the data, and returns a dictionary that matches the `Dataset` constructor signature.

We will place this function in `localdata.py`, (and add it to `__all__`) to make it visible to our dataset code.

Here's what the `Dataset` constructor signature looks like:

In [23]:
help(Dataset.__init__)

Help on function __init__ in module src.data.dset:

__init__(self, dataset_name=None, data=None, target=None, metadata=None, license_txt=None, descr_txt=None, license_file=None, descr_file=None, **kwargs)
    Object representing a dataset object.
    Notionally compatible with scikit-learn's Bunch object
    
    dataset_name: string (required)
        key to use for this dataset
    data:
        Data: (usually np.array or np.ndarray)
    target: np.array
        Either classification target or label to be used. for each of the points
        in `data`
    metadata: dict
        Data about the object. Key fields include `license_txt` and `descr`
    license_txt: str
        String to use as the LICENSE for this dataset
    license_file: filename
        If `license_txt` is None, license text can be read from this file
    descr_txt: str
        String to use as the DESCR (description) for this dataset
    descr_file: filename
        If `descr_txt` is None, description text can be rea

In [24]:
def process_yelp(dataset_name='yelp', metadata=None, num_reviews=None, filename=None, random_seed=None):
    """Convert raw yelp data to a Dataset Options dictionary
    
    Parameters
    ----------
    num_reviews: int or None
        if set, randomly sample this many reviews from the yelp data
    random_seed: int
        Set for reproducible randomness
    """
    if metadata is None:
        metadata = {}
    if filename is None:
        filename = interim_data_path / 'yelp' / 'yelp_academic_dataset_review.json'
    else:
        filename = pathlib.Path(filename)
    
    if random_seed is not None:
        metadata['random_seed'] = random_seed
        random.seed(random_seed)
    
    if num_reviews is None:
        df = yelp_json_to_df(filename)
    else:
        sample = reservoir_sample(review_file, k, random_seed=random_seed)
        data_json_str = "[" + ','.join(sample) + "]"
        df = pd.read_json(data_json_str)
    
    data = np.array(df.text)
    
    target = None    
    dset_opts = {
        'dataset_name': dataset_name,
        'data': data,
        'target': target,
        'metadata': metadata
    }
    return dset_opts
        

In [25]:
%%time
opts = process_yelp(num_reviews=10000, metadata=dset.metadata)

CPU times: user 30.4 s, sys: 4.22 s, total: 34.6 s
Wall time: 44.1 s


In [26]:
%%file ../src/data/localdata.py
"""
Custom dataset processing/generation functions should be added to this file
"""

from src.data.utils import reservoir_sample
from src.paths import interim_data_path
import pandas as pd
import numpy as np
import random

__all__ = ['process_yelp']

def yelp_json_to_df(filename):
    """Convert a yelp JSON file to a dataframe
    
    Yelp data consists of one json-object per line.
    
    Parameters
    ----------
    
    Returns
    -------
    dataframe corresponding to the file
    """
    with open(filename, 'r') as f:
        data = f.readlines()

    # remove the trailing CR from each line
    data = map(lambda x: x.rstrip(), data)

    # Pack it up as a json list
    data_json_str = "[" + ','.join(data) + "]"

    # and load it into pandas
    df = pd.read_json(data_json_str)
    return df

def process_yelp(dataset_name='yelp', metadata=None, num_reviews=100000, filename=None, random_seed=None):
    """Convert raw yelp data to a Dataset Options dictionary
    
    Parameters
    ----------
    num_reviews: int or None
        if set, randomly sample this many reviews from the yelp data
    random_seed: int
        Set for reproducible randomness
    """
    if metadata is None:
        metadata = {}
    if filename is None:
        filename = interim_data_path / 'yelp' / 'yelp_academic_dataset_review.json'
    else:
        filename = pathlib.Path(filename)
    
    if random_seed is not None:
        metadata['random_seed'] = random_seed
        random.seed(random_seed)
    
    if num_reviews is None:
        df = yelp_json_to_df(filename)
    else:
        sample = reservoir_sample(filename, n_samples=num_reviews, random_seed=random_seed)
        data_json_str = "[" + ','.join(sample) + "]"
        df = pd.read_json(data_json_str)
        metadata['num_reviews'] = num_reviews
    
    data = np.array(df.text)
    
    target = None    
    dset_opts = {
        'dataset_name': dataset_name,
        'data': data,
        'target': target,
        'metadata': metadata
    }
    return dset_opts


Overwriting ../src/data/localdata.py


Let's make sure this works as expected

In [27]:
from src.data.localdata import process_yelp

for k in [100000]:
    dset_opts = process_yelp(num_reviews=k)
    dset = Dataset(**dset_opts)
    print(f'k={k}: data={dset.data.shape}')

k=100000: data=(100000,)


This all looks good


In [28]:
datasets.add_dataset_from_function(dataset_name, process_yelp);

Finally, re-load the dataset and save a copy of it

In [29]:
%%time
ds = load_dataset(dataset_name, num_reviews=99999)
print(str(ds))
ds.dump()

<Dataset: yelp, data.shape=(99999,), metadata=['license', 'descr', 'dataset_name', 'num_reviews', 'hash_type', 'data_hash', 'target_hash']>
CPU times: user 756 ms, sys: 217 ms, total: 973 ms
Wall time: 1.71 s


The saved data is stored in the `processed_data_path`. An copy of just the metadata is also stored, so that it may be quickly checked without loading the entire dataset.

In [30]:
list_dir(processed_data_path)

['.gitkeep', 'yelp.dataset', 'yelp.metadata']

In [31]:
ds.data.shape

(99999,)

In [32]:
ds.metadata.keys()

dict_keys(['license', 'descr', 'dataset_name', 'num_reviews', 'hash_type', 'data_hash', 'target_hash'])

In [33]:
ds = load_dataset(dataset_name, num_reviews=100000)

In [34]:
ds.data.shape

(100000,)

In [35]:
ds.data[0]

"I wouldn't recommend this place to a dog. I can't believe the CEO is a Dragon. He has no business giving advice on anything based on the way this place is run.   The food is disgusting and the service was a joke. No idea what prompted me to come here with so many great places in the neighbourhood."